In [1]:
import os, sys, timeit

import numpy as np
import pandas as pd

from metrics import process

from tqdm import tqdm
from os.path import join
from kmeans_clustering import KMeansClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_boston

import matplotlib.pyplot as plt

%matplotlib inline

In [18]:
pwd = os.getcwd()
data_dir = join(pwd, 'data/')
need_normalization = False
need_standardization = False
need_pca = True
filename = 'cancer.csv'
datapath = join(data_dir, filename)
num_clusters = 5
num_iterations = 300
num_iter_exp = 20
verbose = False
data_boston = load_boston()
do_boston = False

In [19]:
# f = open('data/data.csv')
# lines = f.read()
# print(type(lines))
# new_lines = ''
# for line in lines:
#     new_lines += str(line.encode('utf-8').strip())
# print(len(new_lines))

In [20]:
df = pd.read_csv(
    datapath, 
    low_memory=False,
    encoding='utf-8',
#     sep=',',
#     header=None,
)
if do_boston is True:
    df = pd.DataFrame(
        data_boston.data, 
        columns=data_boston.feature_names,
    )

In [21]:
print(len(df))

105


In [22]:
print(df.head())

  Complete TCGA ID  Gender  Age at Initial Pathologic Diagnosis ER Status  \
0     TCGA-A2-A0T2  FEMALE                                   66  Negative   
1     TCGA-A2-A0CM  FEMALE                                   40  Negative   
2     TCGA-BH-A18V  FEMALE                                   48  Negative   
3     TCGA-BH-A18Q  FEMALE                                   56  Negative   
4     TCGA-BH-A0E0  FEMALE                                   38  Negative   

  PR Status HER2 Final Status Tumor Tumor--T1 Coded Node Node-Coded  ...  \
0  Negative          Negative    T3         T_Other   N3   Positive  ...   
1  Negative          Negative    T2         T_Other   N0   Negative  ...   
2  Negative          Negative    T2         T_Other   N1   Positive  ...   
3  Negative          Negative    T2         T_Other   N1   Positive  ...   
4  Negative          Negative    T3         T_Other   N3   Positive  ...   

   PAM50 mRNA SigClust Unsupervised mRNA SigClust Intrinsic mRNA  \
0  Basal-lik

In [7]:
print(df.dtypes)

eruptions    float64
waiting        int64
dtype: object


In [8]:
columns = df.columns.tolist()
cols = columns
# cols = columns[1:]
# cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Fare']
cols = columns[:-1]
# cols = ['Passenger Count', 'Adjusted Passenger Count']
# cols = columns[1:]
# cols = columns[2:]
df = df[cols]
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.fillna(0)
df = df.replace([-np.inf, np.inf], np.nan)
df = df.dropna()
df = df.loc[:, (df != 0).any(axis=0)]
print(len(df))
print(df.dtypes)

272
eruptions    float64
dtype: object


In [9]:
df = df.sample(
    frac=1.0,
    random_state=1,
)
df = df.reset_index(drop=True)

In [10]:
if need_normalization is True:
    normalizer = StandardScaler()
    tmp = normalizer.fit_transform(df)
    df = pd.DataFrame(tmp, columns=df.columns)
elif need_standardization is True:
    scaler = MinMaxScaler()
    tmp = scaler.fit_transform(df)
    df = pd.DataFrame(tmp, columns=df.columns)

In [11]:
X = df
if need_pca is True:
    pca = PCA(
        n_components=len(cols),
        svd_solver='auto',
    )
    X = pca.fit_transform(X)
else:
    X = X.to_numpy()

In [12]:
for x in X:
    for a in x:
        if np.isinf(a):
            print(x)

In [13]:
data = X
print(data[:5])
print(data.shape)

[[ 0.07078309]
 [-0.11221691]
 [ 1.07078309]
 [-1.24521691]
 [-0.42921691]]
(272, 1)


In [14]:
models = ['coc', 'kmeans++', 'ostrovsky', 'kmeans']
models = ['kmeans++', 'kmeans++_improved']

In [15]:
def plot_scatter(X, labels):
    fig, ax = plt.subplots()
    scatter = plt.scatter(
        X[:, 0],
        X[:, 1],
        c=labels,
    )
    handles, labels = scatter.legend_elements()
    legend = ax.legend(
        handles,
        labels,
        loc='upper right',
        title='Label',
    )
    ax.add_artist(legend)
    plt.title(model+' '+str(num_clusters)+ ' clusters')
    plt.xlabel('Component A')
    plt.ylabel('Component B')
    plt.show()

In [16]:
algorithms = {}
iterations = {}
inertias = {}
times = {}
inertias_min = {}
iters_min = {}
for model in models:
    iterations[model] = 0.0
    inertias[model] = 0.0
    times[model] = 0.0
    iters_min[model] = 0
    inertias_min[model] = 1e20
for i in tqdm(range(num_iter_exp)):
    for model in (models):
        algorithms[model] = KMeansClustering(
            n_clusters=num_clusters,
            max_iter=num_iterations,
            init=model,
            verbose=verbose,
        )
        start = timeit.default_timer()
        algorithm = algorithms[model].fit(data)
        centers = algorithm.cluster_centers_
        labels = algorithms[model].predict(data)
#         plot_scatter(X, labels)
        inertias[model] += algorithm.sse_
        inertias_min[model] = min(inertias_min[model], algorithm.sse_)
        iters_min[model] += algorithm.iter_convergence_
        end = timeit.default_timer()
        times[model] += (end-start)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:36<00:00,  1.83s/it]


In [17]:
for model in models:
    print(model+', '+str(num_clusters))
    print('Average Inertia -', round(inertias[model]/num_iter_exp, 2))
    print('Minimum Inertia - ', round(inertias_min[model], 2))
    print('Time -', round(times[model]/num_iter_exp, 2))
#     print('Average number of iterations', round(iters_min[model]/num_iter_exp,2))
    print('')

kmeans++, 5
Average Inertia - 7.66
Minimum Inertia -  7.0
Time - 0.91

kmeans++_improved, 5
Average Inertia - 7.55
Minimum Inertia -  7.0
Time - 0.91

